In [1]:
import pandas as pd
import redcap
import matplotlib
import datetime
data = {
   'token': 'DE12CA04415CF93BCD68321EBF5040FD',
    'content': 'project',
    'format': 'json',
    'returnFormat': 'json'
}
df= redcap.Project('https://redcap.mrc.gm:8443/redcap/api/',data['token'])


data = {
    'token': 'B5C8DD01C47A66E81E154A8AC735D281',
    'content': 'project',
    'format': 'json',
    'returnFormat': 'json'
}
ck_wk=redcap.Project('https://redcap.mrc.gm:8443/redcap/api/',data['token'])

data = {
    'token': 'D6807C091B1FC14F57CC9B2972D0B398',
    'content': 'project',
    'format': 'json',
    'returnFormat': 'json'
}
indigo= redcap.Project('https://ruff.mrc.gm:8443/redcap/api/', data['token'])

data = {
    'token': '9E57344030A7BEF8E9477ADB856CDD63',
    'content': 'project',
    'format': 'json',
    'returnFormat': 'json'
}
eden_test =redcap.Project('https://redcap.mrc.gm:8443/redcaptest/api/',data['token'])


# Documentation for REDCap Data Processing Scripts

## Overview
This Jupyter Notebook contains scripts to export, process, and import data between different REDCap projects. The data is primarily related to maternal and infant health, including deliveries, maternal health checks, baby checks, ultrasound scans, obstetric and medical history, and anthropometry.

## Cell Descriptions

### Cell 0
- **Imports**: pandas, redcap, matplotlib, datetime
- **REDCap Projects**: Initializes connections to multiple REDCap projects using API tokens.

### Cell 1
- **Export Records**: Exports records from the Indigo database.
- **Data Processing**: Filters and renames columns, converts the DataFrame to a list of dictionaries.
- **Import Records**: Imports processed records into the eden_test database.

### Cell 2
- **Export Records**: Exports delivery records from the Indigo database.
- **Data Processing**: Filters, merges, and selects specific columns.
- **Import Records**: Imports processed delivery records into the eden_test database.

### Cell 3
- **Export Records**: Exports maternal health check records from the Indigo database.
- **Data Processing**: Filters, merges, and selects specific columns.
- **Import Records**: Imports processed maternal health check records into the eden_test database.

### Cell 4
- **Export Records**: Exports baby check records from the Indigo database.
- **Data Processing**: Filters, merges, converts columns to numeric, computes averages, renames columns, and selects specific columns.
- **Export to CSV**: Saves the processed baby check data to a CSV file.

### Cell 5
- **Export Records**: Exports ultrasound scan records from the Indigo database.
- **Data Processing**: Filters, renames columns, maps weeks, and selects specific columns.
- **Import Records**: Imports processed ultrasound scan records into the eden_test database.

### Cell 6
- **Export Records**: Exports obstetric and medical history records from the Indigo database.
- **Data Processing**: Filters, renames columns, and selects specific columns.
- **Export to CSV**: Saves the processed obstetric and medical history data to a CSV file.

### Cell 7
- **Export Records**: Exports maternal anthropometry and vital signs records from the Indigo database.
- **Data Processing**: Filters, merges, renames columns, maps weeks, and selects specific columns.
- **Import Records**: Imports processed maternal anthropometry records into the eden_test database.

### Cell 8
- **Export Records**: Exports infant anthropometry records from the Indigo database.
- **Data Processing**: Filters, merges, renames columns, maps months, and selects specific columns.
- **Import Records**: Imports processed infant anthropometry records into the eden_test database.

## Notes
- The scripts use the `pandas` library for data manipulation and the `redcap` library for interacting with REDCap projects.
- Data is exported from the Indigo database and imported into the eden_test database after processing.
- Some cells save the processed data to CSV files for further analysis or backup.

In [2]:
# Export records from Indigo database
df_incons = indigo.export_records(fields=('con_participantid_q1', 'con_participant_eden_num_q11'))

# Convert to DataFrame
df_incons = pd.DataFrame(df_incons)

# Filter out rows where 'con_participant_eden_num_q11' is empty
df_incons = df_incons[df_incons['con_participant_eden_num_q11'] != '']

# Rename 'con_participant_eden_num_q11' to 'participant_id'
df_incons = df_incons.rename(columns={'con_participant_eden_num_q11': 'participant_id'})

# Select only the 'participant_id' column
df_incons_main=df_incons[['con_participantid_q1','participant_id']]
df_incons = df_incons[['participant_id']]

# Convert DataFrame to a list of dictionaries (required for REDCap import)
records_to_import = df_incons.to_dict(orient='records')

# Import records into the eden_test database
respons = eden_test.import_records(records_to_import, return_content='ids', overwrite=False)


In [3]:
#deveries data.
deliveries=indigo.export_records(forms=['delivery'])
deliveries=pd.DataFrame(deliveries)
deliveries=deliveries[
    (deliveries['del_vdate_q1']!='')
]
df_delveries_data=pd.merge(deliveries,df_incons_main,on='con_participantid_q1',how='inner')
df_delveries_data['redcap_event_name']='delivery_and_postp_arm_1'

df_delveries_data=df_delveries_data[[ 'participant_id','redcap_event_name', 'redcap_repeat_instrument',
       'redcap_repeat_instance', 'del_vdate_q1', 'del_place_q2',
       'del_delplaceother', 'del_attendant_q3', 'del_lbonsetdatetime_q4',
       'del_lblength_q5', 'del_presentation_q6', 'dlv_other_presentation_q6b',
       'del_ddate_q7', 'del_delmode_q8', 'del_outcome_q9', 'del_gestage',
       'del_sex_q10', 'del_birthweight_avai_q11', 'del_birthweight_q11a',
       'del_problems_q12', 'del_1mapscorerec_q13',
       'del_1mactivitymusctone_q14', 'del_1mpulserate_q15',
       'del_1mgrimace_q16', 'del_1mappearance_q17', 'del_1mrespiration_q18',
       'del_1mapgarscore_q19', 'del_5mapscorerec_q20',
       'del_5mactivitymusctone_q21', 'del_5mpulserate_q22',
       'del_5mgrimace_q23', 'del_5mappearance_q24', 'del_5mrespiration_q25',
       'del_5mapgarscore_q26', 'del_completed_by_q27', 'del_verified_by_q28',
       'delivery_complete']]
deveries_to_import = df_delveries_data.to_dict(orient='records')

# Import records into the eden_test database
respons = df.import_records(deveries_to_import, return_content='ids', overwrite=False)

In [4]:
#maternal health check
maternal_health_check=indigo.export_records(forms=['maternal_health_check'])
maternal_health_check=pd.DataFrame(maternal_health_check)
maternal_health_check=maternal_health_check[
    (maternal_health_check['maternal_health_check_complete']!='')
]
maternal_health_check_data=pd.merge(maternal_health_check,df_incons_main,on='con_participantid_q1',how='inner')
maternal_health_check_data['redcap_event_name']='delivery_and_postp_arm_1'

maternal_health_check_data=maternal_health_check_data[[ 'participant_id','redcap_event_name', 'redcap_repeat_instrument',
       'redcap_repeat_instance', 'symptom_pregnancy', 'mhc_seizures',
       'mhc_vomit_since_delivery_q', 'mhc_had_fever_since_delivery',
       'mch_feel_dizzy_q', 'mhc_had_pain_when_urinating',
       'mhc_had_vaginal_discharge', 'mhc_heavy_bleeding_q',
       'mhc_given_blood_trans_postde', 'mhc_have_lochia_q',
       'mhc_currently_on_med', 'mhc_medications_list', 'mhc_jaundice_q',
       'mhc_pallor', 'mhc_pulse', 'mhc_resp_rate', 'mhc_oxygen_saturation',
       'mhc_temperature', 'mhc_systolic_bp', 'mhc_diastolic_bp',
       'mhc_systolicbp', 'mhc_diastolicbp', 'mhcsystolicbp', 'mhc_diastolicb',
       'mhc_weight', 'hc_comment', 'mhc_completeby', 'mhc_date_complete',
       'verify_bymh', 'maternal_health_check_complete']]

maternal_health_check_to_import = maternal_health_check_data.to_dict(orient='records')

# Import records into the eden_test database
respons = df.import_records(maternal_health_check_to_import, return_content='ids', overwrite=False)



In [5]:
# Export records from the REDCap 'baby_check' form
baby_check = indigo.export_records(forms=['baby_check'])
baby_check = pd.DataFrame(baby_check)

# Filter out rows where 'bc_date' is empty
baby_check = baby_check[baby_check['bc_date'] != '']

# Merge the baby_check data with df_incons_main on 'con_participantid_q1'
baby_check_data = pd.merge(baby_check, df_incons_main, on='con_participantid_q1', how='inner')
baby_check_data['redcap_event_name'] = 'delivery_and_postp_arm_1'
baby_check_data['redcap_repeat_instrument'] = 'baby_check'
baby_check_data['redcap_repeat_instance'] = '1'

# Columns to convert to numeric
numeric_columns = [
    'bc_temperature','bc_oxsat', 'bc_resprate', 'bc_heartrate', 
    'bc_weight1', 'bc_weight2', 'bc_weight3', 'bc_birthlength1', 
    'bc_birthlength2', 'bc_birthlength3', 'bc_headcf1', 'bc_headcf2', 
    'bc_headcf3', 'bc_muac1', 'bc_muac2', 'bc_muac3'
]

# Convert specified columns to numeric, coercing errors to NaN
for col in numeric_columns:
    if col in baby_check_data.columns:
        baby_check_data[col] = pd.to_numeric(baby_check_data[col], errors='coerce')

# Compute averages and round to appropriate precision
baby_check_data['bc_birthweight'] = (
    baby_check_data[['bc_weight1', 'bc_weight2', 'bc_weight3']].mean(axis=1)
).round(2)
baby_check_data['bc_birthlength'] = (
    baby_check_data[['bc_birthlength1', 'bc_birthlength2', 'bc_birthlength3']].mean(axis=1)
).round(1)
baby_check_data['bc_head_circunference'] = (
    baby_check_data[['bc_headcf1', 'bc_headcf2', 'bc_headcf3']].mean(axis=1)
).round(1)
baby_check_data['bc_muac'] = (
    baby_check_data[['bc_muac1', 'bc_muac2', 'bc_muac3']].mean(axis=1)
).round(1)
# Rename columns for REDCap field compatibility
baby_check_data = baby_check_data.rename(columns={
    'bc_date': 'bc_date_of_visit',
    'bc_birthga':'bc_gestational_age',
    'bc_worried': 'bc_worried_about_child',
    'bc_breastfeeding': 'bc_breastfeeding_problem',
    'bc_meconium': 'bc_baby_pass_meconium',
    'bc_passurine': 'bc_baby_pass_urine',
    'bc_vomit': 'bc_baby_vomit',
    'bc_skininfection': 'bc_skin_infection',
    'bc_resprate': 'bc_respiratory_rate',
    'bc_respdistress': 'bc_respiratory_distress',
    'bc_sunkantfont': 'bc_sunken_anterior_fon',
    'bc_heartrate': 'bc_heart_rate',
    'bc_temperature': 'bc_temperature',
    'bc_fempulse': 'bc_femoral_pulses_present',
    'bc_alertness': 'bc_alertness',
    'bc_reflexes': 'bc_reflexes',
    'bc_tone': 'bc_tone',
    'bc_dysmorphic': 'bc_dysmorphic_features',
    'bc_abpalate': 'bc_abdormal_palate',
    'bc_abdomdist': 'bc_abnormal_distention',
    'bc_anuspatent': 'bc_anus_patent',
    'bc_spine': 'bc_spine_normal',
    'bc_oralthrush': 'bc_oral_thrush',
    'bc_eyes': 'bc_eyes_normal',
    'bc_referral': 'bc_referal_to_keneba',
    'bc_completedby': 'baby_check_completedby_q34',
    'baby_check_complete': 'baby_check_complete',
    'bc_drcomments': 'bc_comments'
})

# Select the columns required for REDCap
baby_check_data = baby_check_data[['participant_id','redcap_event_name', 'redcap_repeat_instrument',
       'redcap_repeat_instance','bc_date_of_visit', 'bc_birthweight', 
    'bc_birthlength', 'bc_head_circunference', 'bc_muac', 'bc_worried_about_child',
    'bc_breastfeeding_problem', 'bc_baby_pass_meconium', 'bc_baby_pass_urine', 
    'bc_baby_vomit', 'bc_skin_infection', 'bc_respiratory_rate', 
    'bc_respiratory_distress', 'bc_sunken_anterior_fon', 'bc_heart_rate',
    'bc_temperature', 'bc_femoral_pulses_present', 'bc_alertness',
    'bc_reflexes', 'bc_tone', 'bc_dysmorphic_features',
    'bc_abdormal_palate', 'bc_abnormal_distention', 'bc_anus_patent',
    'bc_spine_normal', 'bc_oral_thrush', 'bc_eyes_normal', 
    'bc_referal_to_keneba','baby_check_complete'
]]
baby_check_data.to_csv('baby_check_data.csv',index=False)

In [7]:
df_ind_ul=indigo.export_records(forms=['ultrasound_scan'])
df_ind_ul=pd.DataFrame(df_ind_ul)
df_ind_ul=df_ind_ul[
    (df_ind_ul['ultrasound_scan_complete']=='2')
]
df_ind_ul['ultrasoun_crownr'] = (df_ind_ul['ga_assessby'] == '1').astype(int)
df_ind_ul['bi_parietal_diameter_cm'] = (df_ind_ul['ga_assessby'] == '2').astype(int)


# Columns to convert to numeric
numeric_columns = [
    
]

# Convert specified columns to numeric, coercing errors to NaN
for col in numeric_columns:
    if col in baby_check_data.columns:
        baby_check_data[col] = pd.to_numeric(baby_check_data[col], errors='coerce')

        

df_ind_ul=df_ind_ul.rename(columns={
    'redcap_event_name':'uss_week',
    'anc_number_q1': 'anc_number',
    'pregnancy_confirmed_by_ult_q2':'pregnancy_confirmed_by_ult',
    'ultsc_cardiac_act_q3':'ultrascan',
    'ultsc_type_q5':'ultrascan_type',
    'ultsc_date':'ultrascan_date',
    'crl': 'ultrascancrl',
    'bi_parietal_diameter_mm':'bi_parietal_diameter_value',
    'ga_weeks':'convert_week_today',
    'ga_days': 'ultrascan_ga_days',
    'hc_measured':'head_circumference_know',
    'hc_m':'head_circumference_value',
    'abdcf_measured':'abdominal_circumference_kn',
    'abdominal_circum_value':'abdominal_circum_value',
    'femoral_length_known': 'femoral_length_known',
    'ultrascan_exp_date': 'ultrascan_exp_date',
    'comments_scan': 'comments_scan',
    'compl':'uss_completedby_q33',
    'upload_uss_report':'upload_uss_report_q35',
    'uss_report_verified_by':'uss_report_verified_by_q36',
    'ultrasound_scan_complete':'ultrasound_scan_complete',
})

weeks_map={
    'recruitment_arm_1':'1',
    'week_20_arm_1':'2',
    'week_28_arm_1':'3',
    'week_36_arm_1':'4',

}
df_ind_ul['uss_week'] = df_ind_ul['uss_week'].map(weeks_map)
df_ind_ul['redcap_repeat_instance'] = df_ind_ul['uss_week']

df_edn_ind_IDs=pd.merge(df_ind_ul,df_incons_main,on='con_participantid_q1',how='inner')
df_edn_ind_IDs['redcap_event_name']='pregnancy_arm_1'
df_edn_ind_IDs['redcap_repeat_instrument']='ultrasound_scan'

records_to_import=df_edn_ind_IDs[['participant_id','redcap_event_name','redcap_repeat_instrument','redcap_repeat_instance' ,'uss_week',
                                'anc_number','ultrascan','ultrascan_date','pregnancy_confirmed_by_ult','ultrascan_type','ultrasoun_crownr','ultrascancrl','bi_parietal_diameter_cm',
                               'bi_parietal_diameter_value','convert_week_today','ultrascan_ga_days',
                               'head_circumference_know','head_circumference_value','abdominal_circumference_kn','abdominal_circum_value',
                               'femoral_length_known','ultrascan_exp_date','comments_scan','upload_uss_report_q35',
                               'ultrasound_scan_complete',
       ]]


combine_record_import = records_to_import.to_dict(orient='records')
respons = eden_test.import_records(combine_record_import, return_content='ids', overwrite=False)

In [8]:
obs_history=indigo.export_records(forms=['obstetric_and_medical_history'])
obs_history=pd.DataFrame(obs_history)
obs_history=obs_history[
    (obs_history['obstetric_and_medical_history_complete']=='2')
]
obs_history=pd.merge(obs_history,df_incons_main,on='con_participantid_q1',how='inner')
obs_history['redcap_event_name']='pregnancy_arm_1'
obs_history=obs_history.rename(columns={
    'omh_screen_hospitalnum_q1_v2':'screen_hospitalnum',
    'omh_age_at_menarche_q2_v2':'do_you_know_the_age_you_at',
    'omh_provide_age_q2a_v2':'provide_for_age',
    'omh_total_pregnancy_q3_v2':'total_pregnancy',
    'omh_livebirths_q4_v2':'have_you_ever_given_birth',
    'stillbirths':'a_how_many_boys',
    'miscarriages':'micarriage',
    'gravida_v2':'gravida',
    'para_v2':'para',
    'if_at_least_one_livebirth_v2':'if_at_least_one_livebirth',
    'have_you_used_contraceptio_v2':'have_you_used_contraceptio',
    'a_what_method_did_you_use_v2':'a_what_method_did_you_use',
    'a_what_method_did_you_use_v2___1':'a_what_method_did_you_use___1',
    'a_what_method_did_you_use_v2___2':'a_what_method_did_you_use___2',
    'a_what_method_did_you_use_v2___3':'a_what_method_did_you_use___3',
    'a_what_method_did_you_use_v2___4':'a_what_method_did_you_use___4',
    'a_what_method_did_you_use_v2___5':'a_what_method_did_you_use___5',
    'a_what_method_did_you_use_v2___6':'a_what_method_did_you_use___6',
    'a_what_method_did_you_use_v2___7':'a_what_method_did_you_use___7',
    'a_what_method_did_you_use_v2___8':'a_what_method_did_you_use___8',
    'a_what_method_did_you_use_v2___9':'a_what_method_did_you_use___9',
    'a_what_method_did_you_use_v2___10':'a_what_method_did_you_use___10',
    'a_what_method_did_you_use_v2___11':'a_what_method_did_you_use___11',
    'a_what_method_did_you_use_v2___12':'a_what_method_did_you_use___12',
    'b_would_you_have_used_cont_v2':'b_would_you_have_used_cont',
    'have_you_ever_been_told_by_v2':'have_you_ever_been_told_by_916d92',
    'have_you_ever_been_treated_v2':'have_you_ever_been_treated_3d961d',
    'you_ever_been_told_by_v2':'you_ever_been_told_by_21b324',
    'have_you_ever_told_by_v2':'have_you_ever_told_by_a72a40',
    'have_you_ever_had_seizures_v2':'have_you_ever_had_seizures_200d6b',
    'do_you_have_any_other_medi_v2':'do_you_have_any_other_medi',
    'please_provide_further_det_v2':'please_provide_further_det_3f0c98',
    'screen36_taking_meds_v2':'screen36_taking_meds',
    'screen37_family_disease_v2':'screen37_family_disease_27e7ef',
    'screen38_diabetes_v2':'screen38_diabetes_4bdfcb',
    'screen39_hypertension_v2':'screen39_hypertension_0e64ed',
    'screen_teenage_pregnancy_v2':'screen_teenage_pregnancy',
    'screen42_elderly_v2':'screen42_elderly',
    'screen45_lga_baby_v2':'screen45_lga_baby',
    'screen_sgababy_v2':'screen_sgababy',
    'screen_twins_v2':'screen_twins',
    'screen_reviewer_v2':'screen_reviewer_65885d',
    'obstetric_and_medical_history_complete':'obstetric_medicalhistory_complete'

})
obs_history=obs_history[['participant_id','redcap_event_name', 'redcap_repeat_instrument',
'redcap_repeat_instance','screen_hospitalnum',
   'do_you_know_the_age_you_at',
    'provide_for_age',
    'total_pregnancy',
    'have_you_ever_given_birth',
    'a_how_many_boys',
    'micarriage',
    'gravida',
    'para',
    'if_at_least_one_livebirth',
    'have_you_used_contraceptio',
    'a_what_method_did_you_use___1',
    'a_what_method_did_you_use___2',
    'a_what_method_did_you_use___3',
    'a_what_method_did_you_use___4',
    'a_what_method_did_you_use___5',
    'a_what_method_did_you_use___6',
    'a_what_method_did_you_use___7',
    'a_what_method_did_you_use___8',
    'a_what_method_did_you_use___9',
    'a_what_method_did_you_use___10',
    'a_what_method_did_you_use___11',
    'a_what_method_did_you_use___12',
    'b_would_you_have_used_cont',
    'have_you_ever_been_told_by_916d92',
    'have_you_ever_been_treated_3d961d',
   'you_ever_been_told_by_21b324',
    'have_you_ever_told_by_a72a40',
    'have_you_ever_had_seizures_200d6b',
    'do_you_have_any_other_medi',
   'please_provide_further_det_3f0c98',
    'screen36_taking_meds',
    'screen37_family_disease_27e7ef',
    'screen38_diabetes_4bdfcb',
    'screen39_hypertension_0e64ed','screen_teenage_pregnancy',
    'screen42_elderly',
    'screen45_lga_baby',
   'screen_sgababy',
   'screen_twins',
'obstetric_medicalhistory_complete']]
obs_history.to_csv('obs_history.csv',index=False)


In [9]:
#mothers Antrops 
df_anthrops=indigo.export_records(forms=['anthropometry_and_vital_signs'])
df_anthrops=pd.DataFrame(df_anthrops)
df_anthrops=df_anthrops[
    (df_anthrops['anthropometry_and_vital_signs_complete']=='2')
]

df_antrops_main=pd.merge(df_incons_main,df_anthrops,on='con_participantid_q1',how='inner')
df_antrops_main['ia_q1_v2']=df_antrops_main['ma_dvdate_q1']
df_antrops_main=df_antrops_main.rename(columns={
    'ma_dvdate_q1':'anthrops_form_date_q11',
    'ma_weight1':'ia_weight2',
    'ma_weight2':'ia_weight3',
    'ma_weight3':'ia_weight4',
    'anthropometry_and_vital_signs_complete':'anthrops_complete'
})
df_week={
    'week_20_arm_1':'31',
    'week_28_arm_1':'32',
    'week_36_arm_1':'33'
}
df_instance={
    '31':'31',
    '32':'32',
    '33':'33'
}
df_instance_repeat={
    '31':'2',
    '32':'3',
    '33':'4'
}
df_antrops_main['redcap_event_name'] = df_antrops_main['redcap_event_name'].map(df_week)
df_antrops_main['cyclea'] = df_antrops_main['redcap_event_name'].map(df_instance)
df_antrops_main['redcap_repeat_instance'] = df_antrops_main['redcap_event_name'].map(df_instance_repeat)
df_antrops_main['redcap_repeat_instrument'] = 'anthrops'
df_antrops_main['redcap_event_name'] = 'pregnancy_arm_1'
df_antrops_main=df_antrops_main[['participant_id', 'redcap_event_name',
        'redcap_repeat_instrument', 'redcap_repeat_instance', 'ia_q1_v2','anthrops_form_date_q11','cyclea','ia_weight2', 'ia_weight3', 'ia_weight4','anthrops_complete'
      ]]
df_antrops_main=df_antrops_main[
    (df_antrops_main['cyclea'].notnull())
]

df_antrops_main_record_import = df_antrops_main.to_dict(orient='records')
respons = df.import_records(df_antrops_main_record_import, return_content='ids', overwrite=False)

In [10]:
df_infant_antrops=indigo.export_records(forms=['infant_anthropometry'])
df_infant_antrops=pd.DataFrame(df_infant_antrops)
df_infant_antrops=df_infant_antrops[
    (df_infant_antrops['infant_anthropometry_complete']=='2')
]
df_infant_antrops_main=pd.merge(df_incons_main,df_infant_antrops,on='con_participantid_q1',how='inner')
months_map={
    'month_1_arm_2':'1',
    'month_2_arm_2':'2',
    'month_3_arm_2':'3',
    'month_4_arm_2':'4',
    'month_5_arm_2':'5',
    'month_6_arm_2':'6',
    'month_7_arm_2':'7',
    'month_8_arm_2':'8',
    'month_9_arm_2':'9',
    'month_10_arm_2':'10',
    'month_11_arm_2':'11',
    'month_12_arm_2':'12'
}

df_infant_antrops_main['redcap_event_name'] = df_infant_antrops_main['redcap_event_name'].map(months_map)
df_infant_antrops_main['redcap_repeat_instance'] = df_infant_antrops_main['redcap_event_name']
df_infant_antrops_main['redcap_repeat_instrument'] = 'infant_anthropometry'
df_infant_antrops_main['redcap_event_name'] = 'delivery_and_postp_arm_1'
df_infant_antrops_main['ia_month'] = df_infant_antrops_main['redcap_repeat_instance']
df_infant_antrops_main=df_infant_antrops_main.rename(columns={ 
    'ian_date':'ia_q1',
    'ian_length1':'ia_length_q2',
    'ian_length2':'ia_length_q3',
    'ian_weight1':'ia_weight1_q4',
    'ian_weight2':'ia_weight2_q5',
    'ian_weight3':'ia_weight3_q6',
    'ian_hc1':'ia_hc1_q7',
    'ian_hc2':'ia_hc2_q8',
    'ian_hc3':'ia_hc3_q9',
    'ian_muac1':'ia_muac1_q10',
    'ian_muac2':'ia_muac2_q11',
    'ian_muac3':'ia_muac3_q12',
    'ian_comments':'ia_comments_q13',

     })
df_infant_antrops_main['ianthrops_from_date_q16']=df_infant_antrops_main['ia_q1']
df_infant_antrops_main=df_infant_antrops_main[['participant_id', 'redcap_event_name',
       'redcap_repeat_instrument', 'redcap_repeat_instance','ia_month','ia_q1','ia_length_q2','ia_length_q3','ia_weight1_q4','ia_weight2_q5',
       'ia_weight3_q6','ia_hc1_q7','ia_hc2_q8','ia_hc3_q9','ia_muac1_q10','ia_muac2_q11','ia_muac3_q12','ia_comments_q13',
       'ianthrops_from_date_q16','infant_anthropometry_complete']]
df_infant_antrops_main_import = df_infant_antrops_main.to_dict(orient='records')
respons = df.import_records(df_infant_antrops_main_import, return_content='ids', overwrite=False)